In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import lxml.html as lh
import re

from sqlalchemy import create_engine


In [2]:

engine = create_engine('sqlite://', echo=False)

In [3]:
states= pd.read_csv('states.csv')
# states=states.rename(columns={"State.1": "StateAbv"})\
#     .drop(columns=['State Capital','Region'])\
#     .reset_index(drop=True)
# Just to be sure no spaces remain in column for merge!!! 
states['State.1'] = states['State.1'].str.strip()
# states.convert_dtypes()
states.head()

,State,State.1,State Capital,Region
0,Alabama,AL,Montgomery,South
1,Alaska,AK,Juneau,West
2,Arizona,AZ,Phoenix,West
3,Arkansas,AR,Little Rock,South
4,California,CA,Sacramento,West


In [11]:
state =states["State.1"]
nat_park=[]  #This is where it all winds up

for name in state:
    st_park = [] # will be a list for list for all st parks- resets on each loop
    
    url=f'https://stateparks.com/{name}.html'
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    results= soup.find('div', id="overview")
    body = results.find_all('div', style="display:inline-block;")
    
    for row_num in range(len(body)): 
        row = [] # this will hold entries for one row- resets each loop
        for row_item in body[row_num]:
            aa = re.sub("(\xa0)|(\n)|,|•| "," ",row_item.text)
            #assign value to the parks where there is just one, scraping give an empty value
            if aa == '':
                aa = 1
            if len(row) >1:
                del row[0]    
            row.append(aa)
            
            def Convert(a):#This will create a dictionary per park with its count 
            #First create an iterator, and initialize it to variable ‘it’
                it = iter(a)
            #Then use zip method, to zip keys and values together. 
            #Finally typecast it to dict type. 
                res_dct = dict(zip(it, it))
            #flip the key value pair so type comes before count
                res_dct2= {v: k for k,v in res_dct.items()}
                return res_dct2
            
             #send park to the Convert function 
        park_dict = Convert(row)
        st_park.append(park_dict)
  
    nat_park.append(st_park)

In [12]:
st_park

[{'National Memorial': 1}, {' National Historic Sites': '2'}]

In [16]:
df=pd.DataFrame(nat_park)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,{' State Parks': '24'},{' State Forests': '3'},{' State Wildlife Management Areas': '4'},{'National Park': 1},{' National Forests': '4'},{' National Historic Sites': '2'},{'National Historic Park': 1},{' National Monuments': '2'},{' National Wildlife Refuges': '10'},None,None,None,None,None,None,None,None,None,None
1,{' State Parks': '4'},{' National Parks': '2'},{' National Forests': '2'},{' National Historic Parks': '2'},{' National Monuments': '2'},{' National Wildlife Refuges': '22'},{'National Recreation Trail': 1},{' National Park and Preserves': '7'},None,None,None,None,None,None,None,None,None,None,None
2,{' State Parks': '16'},{'State Recreation Area': 1},{'State Historic Site': 1},{'State Fish Hatchery': 1},{'State Wildlife Area': 1},{' National Parks': '4'},{' National Forests': '8'},{'National Memorial': 1},{'National Historic Site': 1},{' National Monuments': '9'},{'National Wildlife Area': 1},{' National Wildlife Refuges': '7'},{'National Recreation Area': 1},{' National Recreation Trails': '4'},{'Metro and City Parks': 1},None,None,None,None
3,{' State Parks': '36'},{'State Forest': 1},{'State Historic Site': 1},{' State Fish Hatcherys': '2'},{'State Natural Area': 1},{' State Wildlife Management Areas': '12'},{'National Park': 1},{' National Forests': '3'},{'National Memorial': 1},{' National Historic Sites': '3'},{' National Wildlife Refuges': '9'},{'National Recreation Trail': 1},None,None,None,None,None,None,None
4,{' State Parks': '109'},{' State Recreation Areas': '9'},{' State Forests': '6'},{' State Historic Sites': '5'},{' State Fish Hatcherys': '5'},{'State Nature Preserve': 1},{' State Reserves': '10'},{' State Wildlife Areas': '10'},{' National Parks': '9'},{' National Forests': '19'},{'National Memorial': 1},{' National Historic Sites': '3'},{' National Monuments': '5'},{' National Wildlife Refuges': '31'},{' National Recreation Areas': '3'},{' National Recreation Trails': '6'},{'National Seashore': 1},{'National Park and Preserve': 1},{'Metro and City Parks': 1}
5,{' State Parks': '6'},{' State Forests': '2'},{'State Fish Hatchery': 1},{' State Wildlife Areas': '5'},{' National Parks': '3'},{' National Forests': '11'},{'National Historic Landmark': 1},{'National Monument': 1},{' National Wildlife Refuges': '4'},{' National Recreation Areas': '2'},None,None,None,None,None,None,None,None,None
6,{' State Parks': '53'},{' State Forests': '25'},{' State Wildlife Areas': '2'},{'Metro and City Parks': 1},None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,{' State Parks': '15'},{' State Forests': '2'},{'State Wildlife Area': 1},{' State Wildlife Management Areas': '3'},{' National Wildlife Refuges': '2'},None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,{' State Parks': '114'},{' State Forests': '5'},{'State Memorial': 1},{' State Historic Sites': '17'},{' State Reserves': '3'},{' State Preserves': '19'},{' National Parks': '3'},{' National Forests': '3'},{' National Memorials': '2'},{'National Historic Landmark': 1},{'National Monument': 1},{' National Wildlife Refuges': '20'},{'National Seashore': 1},None,None,None,None,None,None
9,{' State Parks': '53'},{'State Forest': 1},{'State Memorial': 1},{' State Historic Sites': '15'},{'State Nature Preserve': 1},{'State Wildlife Management Area': 1},{'State Wildlife Refuge': 1},{' National Forests': '2'},{'National Historic Site': 1},{' National Wildlife Refuges': '9'},{'National Recreation Area': 1},{'National Seashore': 1},{' Army Corps of Engineerss': '2'},None,None,None,None,None,None


In [17]:
stateparks= df.join(states)
stateparks.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,State,State.1,State Capital,Region
0,{' State Parks': '24'},{' State Forests': '3'},{' State Wildlife Management Areas': '4'},{'National Park': 1},{' National Forests': '4'},{' National Historic Sites': '2'},{'National Historic Park': 1},{' National Monuments': '2'},{' National Wildlife Refuges': '10'},None,...,None,None,None,None,None,None,Alabama,AL,Montgomery,South
1,{' State Parks': '4'},{' National Parks': '2'},{' National Forests': '2'},{' National Historic Parks': '2'},{' National Monuments': '2'},{' National Wildlife Refuges': '22'},{'National Recreation Trail': 1},{' National Park and Preserves': '7'},None,None,...,None,None,None,None,None,None,Alaska,AK,Juneau,West
2,{' State Parks': '16'},{'State Recreation Area': 1},{'State Historic Site': 1},{'State Fish Hatchery': 1},{'State Wildlife Area': 1},{' National Parks': '4'},{' National Forests': '8'},{'National Memorial': 1},{'National Historic Site': 1},{' National Monuments': '9'},...,{' National Recreation Trails': '4'},{'Metro and City Parks': 1},None,None,None,None,Arizona,AZ,Phoenix,West
3,{' State Parks': '36'},{'State Forest': 1},{'State Historic Site': 1},{' State Fish Hatcherys': '2'},{'State Natural Area': 1},{' State Wildlife Management Areas': '12'},{'National Park': 1},{' National Forests': '3'},{'National Memorial': 1},{' National Historic Sites': '3'},...,None,None,None,None,None,None,Arkansas,AR,Little Rock,South
4,{' State Parks': '109'},{' State Recreation Areas': '9'},{' State Forests': '6'},{' State Historic Sites': '5'},{' State Fish Hatcherys': '5'},{'State Nature Preserve': 1},{' State Reserves': '10'},{' State Wildlife Areas': '10'},{' National Parks': '9'},{' National Forests': '19'},...,{' National Wildlife Refuges': '31'},{' National Recreation Areas': '3'},{' National Recreation Trails': '6'},{'National Seashore': 1},{'National Park and Preserve': 1},{'Metro and City Parks': 1},California,CA,Sacramento,West


In [18]:
stateparks.to_csv("stateparks.csv", index = False)